In [ ]:
!pip install --upgrade gspread google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
import gspread
from google.colab import auth
from google.auth.transport.requests import AuthorizedSession
from google.oauth2.credentials import Credentials

# Authenticate the user
auth.authenticate_user()

import google.auth

# Get the authenticated credentials
creds, _ = google.auth.default()

# Authorize the gspread client with the credentials
client = gspread.authorize(creds)


In [ ]:
# Replace with your Google Sheets URL
sheet_url = "https://docs.google.com/spreadsheets/d/1nW7tnebVuUos2Hbuh7uZGPglSqiax9StFKkR1yt4sVQ/edit?gid=0#gid=0"

# Open the Google Sheet
sheet = client.open_by_url(sheet_url).sheet1  # Access the first worksheet

In [54]:
import numpy as np
from scipy.stats import dirichlet
import statistics

def read_parameters():

      our_choice = int(sheet.acell('A7').value)
      total_rolls = 13
      rounds_revealed = int(sheet.acell('A10').value)
      other_choices_probs_str = sheet.acell('A3').value
      other_choices_probs = [float(prob) for prob in other_choices_probs_str.split(',')]
      teams = int(sheet.acell('C3').value)

      rolls_list = []
      for i in range(rounds_revealed):
        rolls_list.append(int(sheet.acell(f'D{7+i}').value))



      # Validate probabilities
      if len(other_choices_probs) != 6:
          raise ValueError("Exactly 6 probabilities required.")
      if not np.isclose(sum(other_choices_probs), 1.0):
          raise ValueError("Probabilities must sum to 1.")
      return our_choice, total_rolls, rounds_revealed, other_choices_probs, rolls_list


def bayesian_update(observed_rolls, prior_alpha=None):
    # Perform Bayesian updating using a Dirichlet prior
    if prior_alpha is None:
        # Use a uniform prior if none is provided
        prior_alpha = np.ones(6)
    # Counts of observed rolls
    observed_counts = np.bincount(observed_rolls, minlength=7)[1:]
    # Posterior alpha parameters
    posterior_alpha = prior_alpha + observed_counts
    # Posterior expected probabilities (mean of Dirichlet distribution)
    posterior_probs = posterior_alpha / posterior_alpha.sum()
    return posterior_probs, posterior_alpha

def calculate_expected_sum(observed_rolls, posterior_probs, total_rolls):
    # Expected value per remaining roll
    expected_value_per_roll = np.dot(np.arange(1, 7), posterior_probs)
    remaining_rolls = total_rolls - len(observed_rolls)
    # Expected total sum
    expected_sum = sum(observed_rolls) + remaining_rolls * expected_value_per_roll
    return expected_sum

def simulate_confidence_interval(observed_rolls, posterior_alpha, total_rolls, confidence_level=95, num_simulations=10000):
    remaining_rolls = total_rolls - len(observed_rolls)
    simulated_totals = []
    # Generate samples from the posterior Dirichlet distribution
    for _ in range(num_simulations):
        # Sample die weights from the posterior Dirichlet distribution
        sampled_probs = dirichlet.rvs(posterior_alpha, size=1)[0]
        # Simulate remaining rolls using sampled probabilities
        simulated_remaining_rolls = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs
        )
        # Calculate total sum
        simulated_total = sum(observed_rolls) + sum(simulated_remaining_rolls)
        simulated_totals.append(simulated_total)
    # Calculate confidence interval
    lower_bound = np.percentile(simulated_totals, (100 - confidence_level) / 2)
    upper_bound = np.percentile(simulated_totals, 100 - (100 - confidence_level) / 2)
    return lower_bound, upper_bound

def main(teams, total_rolls, our_choice, rounds_revealed, other_choices_probs, observed_rolls):

    posterior_probs, posterior_alpha = bayesian_update(observed_rolls)

    # Step 5: Calculate expected sum based on updated die weights
    expected_sum = calculate_expected_sum(observed_rolls, posterior_probs, total_rolls)

    # Step 6: Simulate confidence interval
    lower_bound95, upper_bound95 = simulate_confidence_interval(
        observed_rolls, posterior_alpha, total_rolls, confidence_level=95, num_simulations=10000
    )

    lower_bound80, upper_bound80 = simulate_confidence_interval(
        observed_rolls, posterior_alpha, total_rolls, confidence_level=80, num_simulations=10000
    )

    lower_bound60, upper_bound60 = simulate_confidence_interval(
        observed_rolls, posterior_alpha, total_rolls, confidence_level=60, num_simulations=10000
    )


    return expected_sum, lower_bound95, upper_bound95, lower_bound80, upper_bound80, lower_bound60, upper_bound60




teams = 60
our_choice, total_rolls, rounds_revealed, other_choices_probs, observed_rolls = read_parameters()
expected_sum, lower_bound95, upper_bound95, lower_bound80, upper_bound80, lower_bound60, upper_bound60 = main(teams, total_rolls, our_choice, rounds_revealed, other_choices_probs, observed_rolls)
print(f"Expected Sum: {expected_sum:.2f}")
print(f"95%: [{lower_bound95:.2f}, {upper_bound95:.2f}]")
print(f"95%: [{lower_bound80:.2f}, {upper_bound80:.2f}]")


sheet.update([[expected_sum]],'F8')

sheet.update([[lower_bound95]], 'F13')
sheet.update([[upper_bound95]],'H13')

sheet.update([[lower_bound80]],'F19')
sheet.update([[upper_bound80]], 'H19')

sheet.update([[lower_bound60]],'F25')
sheet.update([[upper_bound60]], 'H25')



Expected Sum: 52.08
95%: [41.00, 63.00]
95%: [45.00, 60.00]


{'spreadsheetId': '1nW7tnebVuUos2Hbuh7uZGPglSqiax9StFKkR1yt4sVQ',
 'updatedRange': 'Sheet1!H25',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}